In [152]:
import matplotlib.pyplot as plt
#plt.style.use('classic')
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import matplotlib.patheffects as mpe
import matplotlib
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics


In [153]:
df = pd.read_csv("Youtube_costarica - Sheet1.csv")

In [154]:
df.columns

Index(['ExpID', 'Case', 'Algo', 'timeslot_sec', 'timeslot_mil',
       'InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100', 'TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk', 'Target', 'Stall', 'Target_2'],
      dtype='object')

In [155]:
# g = sns.FacetGrid(df, col="Target_2", hue="Stall")
# g.map(plt.scatter, "timeslot_sec", "TPT")

In [156]:
def Classification(CLASS):
    if CLASS >= 0 and CLASS <= 2:
        return 'Poor'
    elif CLASS > 2 and CLASS <= 3:
        return 'Average'
    elif CLASS >3 and CLASS <= 5:
        return 'Good'

In [157]:
df['Case'].unique()

array(['costa_rica'], dtype=object)

In [158]:
df.columns

Index(['ExpID', 'Case', 'Algo', 'timeslot_sec', 'timeslot_mil',
       'InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100', 'TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk', 'Target', 'Stall', 'Target_2'],
      dtype='object')

In [159]:
nan_cols = [i for i in df.columns if df[i].isnull().any()]

In [160]:
nan_cols

[]

In [161]:
from sklearn import preprocessing


In [225]:
X = df[['InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100', 'TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk']]
y = df['Target_2']

In [226]:
X.columns

Index(['InterIntervalTime', 'AveragePTime', 'AveragePTime_gt100',
       'InterIntervalTime_gt100', 'TPT', 'T_pk', 'T_pk_gt100', '10_per_pk',
       '20_per_pk', '30_per_pk', '40_per_pk', '50_per_pk', '60_per_pk',
       '70_per_pk', '80_per_pk', '90_per_pk'],
      dtype='object')

In [227]:
# X=preprocessing.normalize(X)
X= preprocessing.scale(X)

In [228]:
y.unique()

array(['Medium', 'High', 'Low'], dtype=object)

In [229]:
# X.shape[0]

In [230]:
sm = SMOTE()
X, y = sm.fit_sample(X, y)

In [231]:
# X.shape[0]

In [232]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [233]:
dummy_y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [235]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=16,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(150,kernel_initializer = 'uniform',  activation='relu'))
    model.add(Dense(250,kernel_initializer = 'uniform',  activation='relu'))

    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



In [236]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=500, batch_size=5, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline: 73.30% (2.49%)


In [237]:
print(results)

[0.72535211 0.71830988 0.77464789 0.74468082 0.70212764]


In [238]:
# estimator.fit(X_train, y_train)
# predictions = estimator.predict(X_test)
#print(predictions)

In [239]:
# print(encoder.inverse_transform(predictions))

In [240]:
X_train, X_test, Y_train, Y_test = train_test_split(X, dummy_y, test_size=0.3)
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

In [241]:
print(predictions[0])
print(encoder.inverse_transform(predictions))

1
['Low' 'High' 'Low' 'High' 'Low' 'Medium' 'Low' 'Medium' 'Medium' 'High'
 'Low' 'Low' 'High' 'High' 'High' 'Low' 'Low' 'Low' 'Low' 'Low' 'Medium'
 'Low' 'Medium' 'Low' 'Medium' 'Low' 'Low' 'Low' 'High' 'Low' 'High'
 'Medium' 'High' 'Medium' 'Low' 'Medium' 'High' 'Low' 'High' 'Medium'
 'Low' 'Low' 'Low' 'Medium' 'Low' 'Low' 'Low' 'Medium' 'Low' 'High'
 'Medium' 'Low' 'High' 'Medium' 'High' 'Medium' 'Low' 'Low' 'High' 'Low'
 'Medium' 'Low' 'Medium' 'Medium' 'Medium' 'Medium' 'Low' 'Medium' 'Low'
 'High' 'High' 'High' 'Medium' 'High' 'High' 'Medium' 'High' 'High'
 'Medium' 'Medium' 'Low' 'Low' 'Medium' 'Medium' 'Medium' 'Medium'
 'Medium' 'High' 'High' 'High' 'Low' 'High' 'Low' 'Medium' 'High' 'High'
 'Low' 'Medium' 'Low' 'Low' 'Low' 'High' 'High' 'Low' 'Low' 'Low' 'Medium'
 'Medium' 'High' 'High' 'Low' 'Low' 'Medium' 'Medium' 'High' 'Low'
 'Medium' 'High' 'Medium' 'Low' 'Medium' 'Medium' 'Low' 'Low' 'High'
 'High' 'High' 'Medium' 'High' 'High' 'High' 'High' 'Low' 'Low' 'High'
 'Low' 'M

In [242]:
Y_test[0]

array([0., 1., 0.], dtype=float32)

In [243]:
import numpy as np
rounded_labels=np.argmax(Y_test, axis=1)
rounded_labels[3]
# 2

1

In [244]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(rounded_labels, predictions)
cm

array([[48, 16,  4],
       [19, 53,  8],
       [ 4,  3, 58]])

In [245]:
print(classification_report(rounded_labels, predictions))

              precision    recall  f1-score   support

           0       0.68      0.71      0.69        68
           1       0.74      0.66      0.70        80
           2       0.83      0.89      0.86        65

    accuracy                           0.75       213
   macro avg       0.75      0.75      0.75       213
weighted avg       0.75      0.75      0.74       213



In [246]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
accuracy_score(rounded_labels, predictions)
print('Accuracy is ' + str(round(accuracy_score(rounded_labels, predictions),2)))

Accuracy is 0.75


In [247]:
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [248]:
cmn= np.around(cm / cm.astype(np.float).sum(axis=1, keepdims=True),2) 

In [249]:
cmn

array([[0.71, 0.24, 0.06],
       [0.24, 0.66, 0.1 ],
       [0.06, 0.05, 0.89]])